### Imports

In [140]:
import wfdb
from IPython.display import display
import os
import numpy as np
import time

### Read Records + Create Dictionary

In [141]:
with open('database/RECORDS', 'r') as records:
    raw_contents = records.read()
    record_list = raw_contents.split('\n')
    
patient_dict = {}
sample_length = 60*60# seconds
sample_frequency = 50 # hz
for record in record_list:
    path = 'database/' + record
    signals, fields = wfdb.rdsamp(path, sampto=sample_length * sample_frequency)
    temp_dict = {}
    temp_dict['s'] = signals
    temp_dict['f'] = fields
    patient_dict[record] = temp_dict

# wfdb.plot_wfdb(record=record)

### Preprocessing *2022 Ye Algorithm*

In [142]:
start_old = time.time()
charis1 = patient_dict['charis1']['s']
icp_data_old = charis1[:,2]

w = 60000 # window length, 20 mins default (60000 samples)
st = 3000 # 5% of window length - steps
d = 60 # downsample factor 

i = 1 # while loop counter
while i + w < len(icp_data_old):
    seg = icp_data_old[i:i+w] # window segment

    # Create a mask for values outside the acceptable ICP bound
    for j in range(w):
        if seg[j] > 50 or seg[j] < -5: 
            seg[j] = np.nan 
    m = np.nanmean(seg) # segment mean
    s = np.nanstd(seg) # segment standard deviation
    s_bounds = (m+3*s, m-3*s) # outlier boundaries (+/-3 times standard deviation)
    # For loop checks if values are outside standard deviation bounds (replaced with mean of segment, if yes)
    for k in range(w):
        if seg[k] > s_bounds[0] or seg[k] < s_bounds[1]:
            seg[k] = m
    icp_data_old[i:i+w] = seg
    i += st

d_icp_data_old = icp_data_old[::d]
end_old = time.time()

print("Preprocessing step complete")
print(f"Trial length: {len(icp_data_old)}. Downsampled to: {len(d_icp_data_old)}")
print(f"Time elapsed: {end_old-start_old} seconds")


Preprocessing step complete
Trial length: 180000. Downsampled to: 3000
Time elapsed: 1.0118069648742676 seconds


### Preprocessing *2024 Venkatesh Algorithm*

In [143]:
start = time.time()

charis1 = patient_dict['charis1']['s']
icp_data = charis1[:,2]

w = 60000 # window length, 20 mins default (60000 samples)
st = 3000 # 5% of window length - steps
d = 60 # downsample factor 
u = 50 # upper bound 50mmHg for ICP
l = -5 # lower bound -5mmHg for ICP

i = 1 # while loop counter
while i + w < len(icp_data):
    seg = icp_data[i:i+w] # window segment

    # Create a mask for values outside the acceptable ICP bound
    icp_mask = ((seg > 50) | (seg < -5))
    seg[icp_mask] = np.nan

    m = np.nanmean(seg) # segment mean
    s = np.nanstd(seg) # segment standard deviation
    s_bounds = (m+3*s, m-3*s) # outlier boundaries (+/-3 times standard deviation)

    # Create a mask for values outside the std bounds
    std_mask = ((seg > s_bounds[0]) | (seg < s_bounds[1]))
    seg[std_mask] = m

    # Create a mask to replace nan values with segment mean 
    nan_mask = (np.isnan(seg))
    seg[nan_mask] = m

    # Store modified segment back into ICP data
    icp_data[i:i+w] = seg

    # Step to next segment position
    i += st

# Downsample data by d steps
d_icp_data = icp_data[::d]

end = time.time()

print("Preprocessing step complete")
print(f"Trial length: {len(icp_data)}. Downsampled to: {len(d_icp_data)}")
print(f"Time elapsed: {end-start} seconds")


Preprocessing step complete
Trial length: 180000. Downsampled to: 3000
Time elapsed: 0.035401105880737305 seconds


#### Preprocessing performance analysis

In [144]:
print(f"Preprocessed output match: {np.array_equal(d_icp_data, d_icp_data_old)}")
print(f"The new algorithm provides an increase in efficiency by a factor of : {1/(end-start)/(end_old-start_old)}")

Preprocessed output match: True
The new algorithm provides an increase in efficiency by a factor of : 27.918077365678712
